## 텐서플로우를 활용하여 신경망 구현하기 - 데이터 전 처리

* 마케터로서 광고 비용에 따른 수익률을 신경망을 통해서 예측하고자 합니다.
* FB,TV,Newspaper에 따른 Sales 예측하기

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
np.random.seed(100)
tf.random.set_seed(100)

In [5]:
# 데이터를 DataFrame 형태로 불러 옵니다.
df = pd.read_csv("Advertising.csv")

In [6]:
print('원본 데이터 샘플 :')
print(df.head(),'\n')

원본 데이터 샘플 :
   Unnamed: 0     FB    TV  Newspaper  Sales
0           1  230.1  37.8       69.2   22.1
1           2   44.5  39.3       45.1   10.4
2           3   17.2  45.9       69.3    9.3
3           4  151.5  41.3       58.5   18.5
4           5  180.8  10.8       58.4   12.9 



In [7]:
# 의미없는 변수는 삭제합니다.
df = df.drop(columns=['Unnamed: 0'])

"""
1. Sales 변수는 label 데이터로 Y에 저장하고 나머진 X에 저장합니다.
"""
X = df.drop(columns=['Sales'])
Y = df['Sales']

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3)

In [8]:
"""
2. 학습용 데이터를 tf.data.Dataset 형태로 변환합니다.
   from_tensor_slices 함수를 사용하여 변환하고 batch를 수행하게 합니다.
"""
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y.values))
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)

In [9]:
# 하나의 batch를 뽑아서 feature와 label로 분리합니다.
[(train_features_batch, label_batch)] = train_ds.take(1)

In [10]:
# batch 데이터를 출력합니다.
# bathch size 5니까, 5개 출력
print('\nFB, TV, Newspaper batch 데이터:\n',train_features_batch)
print('Sales batch 데이터:',label_batch)


FB, TV, Newspaper batch 데이터:
 tf.Tensor(
[[296.4  36.3 100.9]
 [228.   37.7  32. ]
 [  5.4  29.9   9.4]
 [ 57.5  32.8  23.5]
 [240.1   7.3   8.7]], shape=(5, 3), dtype=float64)
Sales batch 데이터: tf.Tensor([23.8 21.5  5.3 11.8 13.2], shape=(5,), dtype=float64)


## 텐서플로우를 활용하여 신경망 구현하기 - 모델 구현

* 텐서플로우와 케라스(Keras)를 활용하여 신경망 모델을 구현  
* |케라스는 텐서플로우 내의 신경망 모델 설계와 훈련을 위한 API 입니다. 케라스는 연속적으로(Sequential) 레이어(Layer)들을 쌓아가며 모델을 생성하고, 사이킷런과 같이 한 줄의 코드로 간단하게 학습 방법 설정, 학습, 평가를 진행할 수 있습니다.

In [11]:
"""
1. tf.keras.models.Sequential()를 활용하여 신경망 모델을 생성합니다.
   자유롭게 layers를 쌓고 마지막 layers는 노드 수를 1개로 설정합니다.
"""
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=(3,)),    # FB, TV, Newspaper 3개
    tf.keras.layers.Dense(1)
    ])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                40        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________
None


## 텐서플로우를 활용하여 신경망 구현하기 - 모델 학습

* 텐서플로우와 케라스(Keras)를 활용하여 신경망 모델을 학습  
 
* 텐서플로우와 케라스를 이용해 신경망 모델을 학습하기 위한 함수/메서드  

model.complie(loss='mean_squared_error', optimizer='adam')

* complie() 메서드는 모델을 어떻게 학습할 지에 대해서 설정합니다.  
loss는 회귀에서는 일반적으로 MSE인 ‘mean_squared_error’, 분류에서는 ‘sparse_categorical_crossentropy’ 를 주로 사용합니다.